## 简单说明

v0版本，实现了基于 `Qwen2.5-Coder-7B-Instruct` 的 css 和 js 生成。

In [1]:
import os
import yaml
import re
import gradio as gr
from modelscope import AutoTokenizer, AutoModelForCausalLM
os.environ['GRADIO_ROOT_PATH'] = f"/{os.environ['JUPYTER_NAME']}/proxy/7860"

2025-01-20 16:15:50.644980: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 16:15:50.705498: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 16:15:51.554458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_prefix = "/mnt/workspace"
config_path = os.path.join(file_prefix, "config.yaml")
with open(config_path, "r", encoding="utf-8") as config_file:
    config = yaml.safe_load(config_file)

In [3]:
def parse_response(pattern, response: str) -> str:
    match = re.search(pattern, response, re.S)
    if match:
        return match.group(1).strip()
    else:
        print("c Error: Response error from LLM.")
        return None

In [4]:
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
componentor = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def get_componentor_prompt(file):
    system_content = config["role"]["componentor"]["system_content"]
    with open(os.path.join(file_prefix, "prompt_files/demo0.html"), 'r', encoding='utf-8') as f:
        demo_source_code = f.read()
    with open(os.path.join(file_prefix, "prompt_files/demo0.js"), 'r', encoding='utf-8') as f:
        demo_js = f.read()
    with open(os.path.join(file_prefix, "prompt_files/demo0.css"), 'r', encoding='utf-8') as f:
        demo_css = f.read()
    demo_prompt = config["role"]["componentor"]["example"].format(demo_source_code=demo_source_code, demo_css=demo_css, demo_js=demo_js)
    system_content = system_content + "\n\n" + demo_prompt
    user_content = config["role"]["componentor"]["user_content"]
    with open(file, "r", encoding="utf-8") as f:
        source_code = f.read()
    user_content = user_content.format(source_code=source_code)

    return system_content, user_content


# 处理上传文件并生成代码
def generate_css_and_js(image, file):
    # 构造 LLM 输入
    system_content, user_content = get_componentor_prompt(file)
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    # 生成 组件化 代码
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(componentor.device)

    generated_ids = componentor.generate(
        **model_inputs,
        max_new_tokens=8192
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # 解析大模型输出，拆分成 js + css
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    css = parse_response(pattern=r"\[\[## css ##\]\]\n(.*?)\n\[\[## js ##\]\]", response=response)
    js = parse_response(pattern=r"\[\[## js ##\]\]\n(.*?)\n\[\[## completed ##\]\]", response=response)
    return css, js

2025-01-20 16:15:54,112 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct].
2025-01-20 16:15:54,113 - modelscope - WARNING - Failed to create symbolic link /mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct for /mnt/workspace/.cache/modelscope/Qwen/Qwen2___5-Coder-7B-Instruct.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-01-20 16:16:28,519 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct].
2025-01-20 16:16:28,519 - modelscope - WARNING - Failed to create symbolic link /mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct for /mnt/workspace/.cache/modelscope/Qwen/Qwen2___5-Coder-7B-Instruct.


In [5]:
# 构建 Gradio 界面
with gr.Blocks(css=".file-input { height: 30px !important; }") as demo:
    gr.Markdown("<center><h1>组件复制门</h1></center>")
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 输入您想提取的组件截图和源码")
            image_input = gr.Image(label="上传图片(png文件)", type="filepath")
            file_input = gr.File(label="上传文件", interactive=True, container=False,
                                 elem_classes="file-input")
            submit_btn = gr.Button("提交")

        with gr.Column(scale=1):
            gr.Markdown("### 封装组件表现效果")
            # python_output = gr.Textbox(label="Python 运行结果", interactive=False)

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 封装的 JS 代码")
            generated_ans_js = gr.Code(label="JavaScript 代码", language="javascript")

        with gr.Column():
            gr.Markdown("### 封装的 CSS 代码")
            generated_ans_css = gr.Code(label="CSS 代码", language="css")

    # 绑定交互逻辑
    submit_btn.click(generate_css_and_js, inputs=[image_input, file_input],
                     outputs=[generated_ans_css, generated_ans_js])

# 运行 Gradio 应用
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 2047, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 1594, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2505, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", 